In [1]:
import pandas as pd

In [2]:
import seaborn as sns

In [3]:
import matplotlib.pyplot as plt

In [4]:
import plotly.graph_objs as go

In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [6]:
init_notebook_mode(connected=True)

In [7]:
bechdel_df = pd.read_csv('my_data/updated_bechdel_4.csv')
bechdel_df.head()

,year,title,score,passing,imdb_id,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
0,2019.0,Alita: Battle Angel,3.0,1.0,tt0437086,0.0,1.0,0.0,0.0,1.0,170000000.0,359700000.0
1,2019.0,Close,3.0,1.0,tt5316540,1.0,1.0,0.0,0.0,2.0,0.0,0.0
2,2019.0,Cold Pursuit,1.0,0.0,tt5719748,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019.0,The Kid Who Would Be King,1.0,0.0,tt6811018,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,2019.0,Polar,2.0,0.0,tt4139588,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
no_title_id_df = bechdel_df[['year', 'passing', 'budget', 'revenue']]
no_title_id_df.head()

,year,passing,budget,revenue
0,2019.0,1.0,170000000.0,359700000.0
1,2019.0,1.0,0.0,0.0
2,2019.0,0.0,0.0,0.0
3,2019.0,0.0,0.0,0.0
4,2019.0,0.0,0.0,0.0


In [10]:
passing_movies = no_title_id_df[no_title_id_df.passing == 1.0]
passing_movies.tail()

,year,passing,budget,revenue
7607,2014.0,1.0,0.0,1358561.0
7608,2013.0,1.0,0.0,0.0
7609,2013.0,1.0,16000000.0,5900000.0
7610,2013.0,1.0,0.0,0.0
7611,2012.0,1.0,0.0,0.0


In [11]:
year_grouped = passing_movies.groupby('year')
year_grouped.head()

,year,passing,budget,revenue
0,2019.0,1.0,170000000.0,3.597000e+08
1,2019.0,1.0,0.0,0.000000e+00
6,2018.0,1.0,0.0,0.000000e+00
10,2018.0,1.0,20000000.0,4.260000e+07
11,2018.0,1.0,36000000.0,4.035881e+08
12,2018.0,1.0,100000000.0,1.326759e+08
17,2018.0,1.0,40000000.0,4.307092e+07
117,2017.0,1.0,5500000.0,4.423502e+07
119,2017.0,1.0,28000000.0,1.305604e+08
120,2017.0,1.0,0.0,0.000000e+00


In [12]:
bechdel_df.groupby('passing').mean()

,year,score,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
passing,,,,,,,,,
0.0,1991.026761,1.000615,0.035681,0.083359,0.015995,0.123962,0.258997,2.116392e+07,6.638363e+07
1.0,1998.382940,3.000000,0.141252,0.224719,0.026141,0.174272,0.566155,1.657806e+07,5.787114e+07


In [15]:
bechdel_df.groupby('passing').sum()

,year,score,director_gender,writer_gender,exec_gender,producer_gender,overall_gender,budget,revenue
passing,,,,,,,,,
0.0,6472828.0,3253.0,116.0,271.0,52.0,403.0,842.0,6.880390e+10,2.158132e+11
1.0,8714948.0,13083.0,616.0,980.0,114.0,760.0,2469.0,7.229692e+10,2.523760e+11


In [16]:
passing = bechdel_df.passing.value_counts()

In [18]:
import cufflinks as cf

ImportError: cannot import name 'FigureWidget' from 'plotly.graph_objs' (/anaconda3/envs/learn-env/lib/python3.7/site-packages/plotly/graph_objs/__init__.py)